In [1]:
import matplotlib
matplotlib.use('Qt5Agg')#for visbrain to work in jupyterlab/notebook
from header import *
import visbrain
from visbrain import Brain, Colorbar
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
import xarray as xr

In [2]:
task = 'SMEG'
names = ['R_ECG_included', 'R_ECG_excluded', 'T_ECG_included', 'T_ECG_excluded']
name = names[0]
noise_cov = 'baseline_cov'
start = 0.35
stop = 0.5
sfreq = 200
path = op.join(Analysis_path, task, 'meg', 'Stats', str(start)+'_'+str(stop))
filename = op.join(path, name+'-'+noise_cov+'.nc')
test_key = 'RS_vs_OM+N'
p_thresh = 0.1

In [3]:
stats = xr.open_dataarray(filename, test_key)
stats.load()

<xarray.DataArray (data: 3, hemisphere: 2, time: 31, src: 10242)>
array([[[[-0.667514, ..., -0.723448],
         ..., 
         [-0.131598, ...,  0.529988]],

        [[-0.737492, ..., -1.294203],
         ..., 
         [-0.188281, ..., -0.538888]]],


       ..., 
       [[[ 0.      , ...,  0.      ],
         ..., 
         [ 0.      , ...,  0.      ]],

        [[ 0.      , ...,  0.      ],
         ..., 
         [ 0.      , ...,  0.      ]]]])
Coordinates:
  * data        (data) object 'T_stat' 'p_val' 'clu_inds'
  * time        (time) float64 0.35 0.355 0.36 0.365 0.37 0.375 0.38 0.385 ...
  * hemisphere  (hemisphere) object 'left' 'right'
  * src         (src) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...

In [4]:
if np.where(stats.loc['p_val', 'left']<p_thresh)[0].any():
    t_signif_lh = np.unique(np.where(stats.loc['p_val', 'left']<p_thresh)[0], return_counts=True)
    best_t_lh = stats.time[t_signif_lh[0][t_signif_lh[1].argsort()][::-1]]
    print(t_signif_lh)
    print(best_t_lh)
else:
    warnings.warn('\nNo significant cluster for left hemisphere.\n')
    best_t_lh = [None]

C:\Users\adorb\WinPython-64bit-3.6.3.0Qt5\python-3.6.3.amd64\lib\site-packages\ipykernel_launcher.py:7: UserWarning: 
No significant cluster for left hemisphere.

  import sys


In [5]:
if np.where(stats.loc['p_val', 'right']<p_thresh)[0].any():
    t_signif_rh = np.unique(np.where(stats.loc['p_val', 'right']<p_thresh)[0], return_counts=True)
    best_t_rh = stats.time[t_signif_rh[0][t_signif_rh[1].argsort()][::-1]]
    print(t_signif_rh)
    print(best_t_rh)
else:
    warnings.warn('\nNo significant cluster for right hemisphere.\n')
    best_t_rh = [None]

(array([10, 11], dtype=int64), array([ 13, 180], dtype=int64))
<xarray.DataArray 'time' (time: 2)>
array([ 0.405,  0.4  ])
Coordinates:
  * time     (time) float64 0.405 0.4


In [6]:
t = float(best_t_rh[0] if best_t_rh[0] is not None else best_t_lh[0])
#t = int(best_t_rh[8])
#t = int(best_t_lh[0])
print('t =', t, 's')
mask_lh = np.where(stats.loc['p_val', 'left', t] < p_thresh)
mask_rh = np.where(stats.loc['p_val', 'right', t] < p_thresh)
lh = np.where(stats.loc['p_val', 'left', t] < p_thresh, stats.loc['T_stat', 'left', t].values, 0)
rh = np.where(stats.loc['p_val', 'right', t] < p_thresh, stats.loc['T_stat', 'right', t].values, 0)

t = 0.405 s


In [7]:
cmax = np.max(np.abs(np.concatenate([lh, rh])))
clim = (-cmax, cmax)

In [8]:
b_obj = BrainObj('inflated', translucent=False, hemisphere='both')#, sulcus=True)

#b_obj.add_activation(data=data_lh[t], vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=data_rh[t], vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_lh[t]>=0.05, -np.inf, data_lh[t]), hide_under=clim[0], vertices=vertices, hemisphere='left', cmap='bwr', clim=clim)
#b_obj.add_activation(data=np.where(pv_lh[t]>=0.05, -np.inf, data_rh[t]), hide_under=clim[0], vertices=vertices, hemisphere='right', cmap='bwr', clim=clim)
b_obj.add_activation(data=lh, vertices=stats.src.values, hemisphere='left', smoothing_steps=4, cmap='bwr', clim=clim)
b_obj.add_activation(data=rh, vertices=stats.src.values, hemisphere='right', smoothing_steps=4, cmap='bwr', clim=clim)


vb = Brain(brain_obj=b_obj)
#cb = Colorbar(vmin=clim[0], vmax=clim[-1], cmap='bwr', ndigits=4, cblabel='T stat')

INFO | BrainObj(name='inflated') created
INFO | Add data to secific vertices.
INFO | Add data to secific vertices.
INFO | VolumeObj(name='brodmann') created
INFO | RoiObj(name='brodmann') created
INFO | brodmann ROI loaded.
INFO | CrossSecObj(name='brodmann') created


In [9]:
vb.show()
vb.cbar_select('brain')
vb.cbar_control('brain', cblabel='{} (t = {} s)'.format(test_key, t))
vb.rotate(custom=(90, 0))
#cb.show()
#sc.preview()

In [10]:
vb.screenshot(op.join(path,'{}_t_{}.png'.format(test_key, t)))

WARNING | dpi parameter is not active if `print_size` is None. Use for example `print_size=(5, 5)`
INFO | Image of size (1761, 1588)px successfully saved (Z:\meditation\ERC\Analyses\SMEG\meg\Stats\0.35_0.5\RS_vs_OM+N_t_0.405.png)


In [ ]:
t_signif_rh